<a href="https://colab.research.google.com/github/abhishek1saini/test_pred/blob/test1/MET34_new_Downsample2T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.8 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta
import tensorflow as tf
import pickle
from collections import defaultdict

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense, TimeDistributed

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,mean_absolute_percentage_error
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from keras_tuner import HyperParameters
from keras_tuner import HyperModel
from keras_tuner.tuners import *

In [5]:
"""## Handling missing, filling with values from previous day"""
def handle_missing(multi_tr_data):
  #multi_tr_data = multi_tr_data.set_index('timestamp')
  for col in multi_tr_data.columns.to_list():
    for header,series in multi_tr_data[multi_tr_data[col].isna()==True].items():
        for dt, value in series.items():
            prev_dt = dt - timedelta(days=1)
            multi_tr_data.loc[dt][col] = multi_tr_data.loc[prev_dt][col]
  return multi_tr_data

def scale_data(train_data, test_data, val_data):
  #scaler = MinMaxScaler()
  scaler = StandardScaler()
  t_train = scaler.fit_transform(train_data)
  t_val = scaler.transform(val_data)
  t_test = scaler.transform(test_data)
  return t_train, t_val, t_test, scaler

def train_test_split(multi_tr_data, test_ratio):
  total_samples = len(multi_tr_data)
  train_samples = int((total_samples)*(1-(test_ratio*2)))
  test_samples=val_samples = int(total_samples*test_ratio)
  train_data = multi_tr_data.iloc[:train_samples].copy(deep=True)
  val_data = multi_tr_data[train_samples:train_samples+val_samples].copy(deep=True)
  test_data = multi_tr_data[train_samples+val_samples:].copy(deep=True)
  return train_data, val_data, test_data

def prep_data(dataset, window, horizon, original_df):
  X, y = [], []
  y_tr_dates = []
  y_ts_dates = []
  series_len = len(dataset)
  end = series_len - horizon
  for win_start in range(series_len):
    x_end = win_start + window
    y_end = x_end + horizon
    if y_end > series_len:
      break
    X.append(dataset[win_start:x_end,:])
    y.append(dataset[x_end:y_end,:])
    y_ts_dates.append(original_df[x_end:y_end].index)
  return np.array(X), np.array(y), np.expand_dims(np.array(y_ts_dates), axis=2)[:,0,:], y_ts_dates

#Preapare the tensors
def prepare_tensors(train: tuple, validation: tuple, batch_size: int, shuffle_buffer_size=1000):
  train_set = tf.data.Dataset.from_tensor_slices(train)
  train_set = train_set.cache().shuffle(shuffle_buffer_size).batch(batch_size)
  val_set = tf.data.Dataset.from_tensor_slices(validation)
  val_set = val_set.cache().shuffle(shuffle_buffer_size).batch(batch_size)
  print(f"Train Tensor {train_set.element_spec}\nValidation Tensor Spec {val_set.element_spec}")
  return train_set, val_set

In [6]:
window_size=320
# BATCH_SIZE=64
features = 2
horizon=30
shuffle_buffer_size = 10000

#raw_data = pd.read_csv('/content/drive/MyDrive/Data/Marjan_ceph_metrics_w8.csv')
raw_data = pd.read_csv('/content/drive/MyDrive/Data/ceph-marjan_custom_timeline_80d_9h.csv')
raw_data = raw_data.tail(50000)
#raw_data = raw_data.drop('Unnamed: 0', axis=1)
raw_data['timestamp'] = pd.to_datetime(raw_data['timestamp'])
raw_data.set_index('timestamp', inplace=True)
multi_data = raw_data[['capacity_instance_ceph_rbd_ops_read_avg', 'capacity_instance_ceph_rbd_ops_write_avg']]
multi_data = multi_data.resample('2T').mean()
#multi_data = multi_data[['date', 'OR', 'OW']]
multi_data.head()

,capacity_instance_ceph_rbd_ops_read_avg,capacity_instance_ceph_rbd_ops_write_avg
timestamp,,
2023-10-25 13:58:00,8.484722,15.562625
2023-10-25 14:00:00,8.815000,16.850598
2023-10-25 14:02:00,8.768964,16.013675
2023-10-25 14:04:00,8.172521,15.147301
2023-10-25 14:06:00,7.014163,13.832236


In [7]:
multi_tr_data = multi_data
# multi_tr_data = multi_data[(multi_data.timestamp >= '2023-05-13 04:18:00') & (multi_data.timestamp <= '2023-05-24')].copy(deep=True)
multi_tr_data = handle_missing(multi_tr_data)
train_data, val_data, test_data,  = train_test_split(multi_tr_data, test_ratio=0.1)
t_train,t_val, t_test, ts_scaler = scale_data(train_data, test_data, val_data)
print(f"Shape of Train set : {t_train.shape}\nShape of Test set : {t_test.shape}\nShape of Validation set : {t_val.shape}")

X_train, y_train, tr_dates, date1 = prep_data(t_train, window=window_size, horizon=horizon, original_df=train_data)
X_val, y_val, val_dates, date2 = prep_data(t_val, window=window_size, horizon=horizon, original_df=val_data)
X_test, y_test, ts_dates, date3  = prep_data(t_test, window=window_size, horizon=horizon, original_df=test_data)
print(f"Shape of X-train : {X_train.shape}, y-train: {y_train.shape}\nShape of X-val : {X_val.shape}, y-val: {y_val.shape}\nShape of X-test : {X_test.shape}, y-test: {y_test.shape}\n")
print("ts_dates for test data: ", type(ts_dates), np.shape(ts_dates))
print(type(date1), type(date2), type(date3))
print(len(date1), len(date2), len(date3))
print("ts_dates:", ts_dates[0])
print(Input(X_train.shape[-2:]))
# train_set, val_set = prepare_tensors(train=(X_train, y_train), validation=(X_val, y_val), batch_size=BATCH_SIZE, shuffle_buffer_size=shuffle_buffer_size)
train_set = (X_train, y_train)
val_set = (X_val, y_val)

Shape of Train set : (20000, 2)
Shape of Test set : (2500, 2)
Shape of Validation set : (2500, 2)
Shape of X-train : (19651, 320, 2), y-train: (19651, 30, 2)
Shape of X-val : (2151, 320, 2), y-val: (2151, 30, 2)
Shape of X-test : (2151, 320, 2), y-test: (2151, 30, 2)

ts_dates for test data:  <class 'numpy.ndarray'> (2151, 1)
<class 'list'> <class 'list'> <class 'list'>
19651 2151 2151
ts_dates: ['2023-11-26T06:38:00.000000000']
KerasTensor(type_spec=TensorSpec(shape=(None, 320, 2), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [8]:
# @title
class seq_HyperModel(HyperModel):
  def build(self, hp):
    tf.keras.backend.clear_session()
    # features = 6
    hidden_units = hp.Int('hidden_units', min_value=64, max_value=512, step=10, sampling='linear')
    dropout_range = hp.Float('dropout', min_value=0.0, max_value=0.2, step=0.1, sampling='linear')
    dropout_enc_0 = hp.Float('dropout_enc_0', min_value=0.0, max_value=0.3, step=0.1, sampling='linear')
    dropout_dec_0 = hp.Float('dropout_dec_0', min_value=0.0, max_value=0.3, step=0.1, sampling='linear')
    kernel_initializer = hp.Choice('kernel_initializer', ['he_uniform'])
    batch_size_range = hp.Int('batch_size', min_value=32, max_value=128)
    learning_rate_range = hp.Float('learning_rate', min_value=1e-6, max_value=1e-3, step=10, sampling='log')
    #learning_rate_range = hp.Choice('learning_rate', [0.00001, 0.000001, 0.0001])
    dense_activation = hp.Choice('dense_activation', ['linear'], default=None)
    optimizer_choice = hp.Choice('optimizer_choice',['adamw', 'sgd'])
    n_layers = hp.Int('n_layers', min_value=1, max_value=6, step=1, sampling='linear')
    momemtum_range = hp.Float('momemtum_range', min_value=0.0, max_value=0.4, step=0.1, sampling='linear')
    loss_func_choice = hp.Choice('loss_function', ['mse', 'huber'])
    input = Input(shape=X_train.shape[-2:])
    lstm1 = LSTM(hidden_units, return_state=True, dropout=dropout_range, kernel_initializer=kernel_initializer)
    st_hidden_, st_hidden, st_cell = lstm1(input)
    repeat = RepeatVector(horizon)
    enc_out = repeat(st_hidden_)
    lstm2 = LSTM(hidden_units, return_sequences=True, dropout=dropout_range,  kernel_initializer=kernel_initializer)
    dec_out = lstm2(enc_out, initial_state=[st_hidden, st_cell])

    with hp.conditional_scope("dense_activation", ["None"]):
      if dense_activation == "None":
        dense = TimeDistributed(Dense(features, activation=None))

    with hp.conditional_scope("dense_activation", ["linear"]):
      if dense_activation == "linear":
        dense = TimeDistributed(Dense(features, activation='linear'))

    final_out = dense(dec_out)
    model = Model(input, final_out)

    with hp.conditional_scope('optimizer_choice',["adam"]):
      if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_range)

    with hp.conditional_scope('optimizer_choice',["adamw"]):
      if optimizer_choice == 'adamw':
        optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate_range)

    with hp.conditional_scope('optimizer_choice',["rmsprop"]):
      if optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate_range, momentum=momemtum_range)

    with hp.conditional_scope('optimizer_choice',["sgd"]):
      if optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate_range, momentum=momemtum_range)
    model.compile(loss=loss_func_choice,
                  # optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_range),
                  optimizer=optimizer,
                  metrics=['mse',tf.keras.metrics.RootMeanSquaredError()])

    return model


  def fit(self, hp, model, train_tuple, val_tuple, *args, **kwargs):
    batch_size_range = hp.Int('batch_size', min_value=8, max_value=128, step=8)
    train_set, val_set = prepare_tensors(train=(X_train, y_train), validation=(X_val, y_val), batch_size=batch_size_range, shuffle_buffer_size=shuffle_buffer_size)
    return model.fit(train_set, validation_data=val_set, *args, **kwargs)

In [9]:
hp = HyperParameters()
bayesian_tuner = BayesianOptimization(
                    hypermodel = seq_HyperModel(),
                    objective = "val_loss",
                    max_trials = 30,
                    #num_initial_points = 20,
                    alpha = 0.0001, # represents the expected amount of noise in the observed performances in Bayesian optimization.
                    beta = 2.6, #  the balancing factor of exploration and exploitation. The larger it is, the more explorative it is
                    hyperparameters = hp,
                    **{"tuner_id" : "BayesianOptimization",
                      "overwrite" : True,
                      "project_name" : "bayesian_optimization"}
                    )
bayesian_tuner.search_space_summary()

Search space summary
Default search space size: 12
hidden_units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 10, 'sampling': 'linear'}
dropout (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.2, 'step': 0.1, 'sampling': 'linear'}
dropout_enc_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}
dropout_dec_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}
kernel_initializer (Choice)
{'default': 'he_uniform', 'conditions': [], 'values': ['he_uniform'], 'ordered': False}
batch_size (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-06, 'conditions': [], 'min_value': 1e-06, 'max_value': 0.001, 'step': 10, 'sampling': 'log'}
dense_activation (Choice)
{'default': 'linear', 'conditions': [], 'values': ['li

In [ ]:
bayesian_tuner.search(train_tuple = train_set, val_tuple = val_set, steps_per_epoch = None, shuffle = False, epochs=20,
                      # validation_split = 0.20,
                      verbose = 1, #
                      use_multiprocessing = True,
                    )
bayesian_tuner.get_best_hyperparameters()[0].values

Trial 19 Complete [00h 05m 22s]
val_loss: 0.7951688766479492

Best val_loss So Far: 0.16051915287971497
Total elapsed time: 01h 30m 25s

Search: Running Trial #20

Value             |Best Value So Far |Hyperparameter
344               |174               |hidden_units
0.1               |0.1               |dropout
0.2               |0                 |dropout_enc_0
0.1               |0.2               |dropout_dec_0
he_uniform        |he_uniform        |kernel_initializer
77                |109               |batch_size
1e-06             |0.001             |learning_rate
linear            |linear            |dense_activation
sgd               |adamw             |optimizer_choice
6                 |6                 |n_layers
0.1               |0.3               |momemtum_range
huber             |huber             |loss_function

Train Tensor (TensorSpec(shape=(None, 320, 2), dtype=tf.float64, name=None), TensorSpec(shape=(None, 30, 2), dtype=tf.float64, name=None))
Validation Tensor Spec

In [ ]:
bayesian_tuner.results_summary(num_trials=10)

In [ ]:
trials = bayesian_tuner.oracle.get_best_trials(num_trials=15)
HP_list = []
for trial in trials:
    HP_list.append(trial.hyperparameters.get_config()["values"] | {"Score": trial.score})
HP_df = pd.DataFrame(HP_list)
#HP_df.to_csv("name.csv", index=False, na_rep='NaN')
#HP_df.to_csv("/content/drive/MyDrive/bayesian_tuner-marjan_25_08_2023.csv", index=False, na_rep='NaN')

In [ ]:
#Build model with best HP
bayesian_best_model = bayesian_tuner.hypermodel.build(bayesian_tuner.get_best_hyperparameters()[0])

In [ ]:
bayesian_best_model.summary()

In [ ]:
#bayesian_best_model.save("/content/drive/MyDrive/bayesian_tuner_marjan_model_25_08_2023")

In [ ]:
HP_df.head()

In [ ]:
params = HP_df.iloc[0]
# hidden_units	dropout	kernel_initializer	batch_size	learning_rate	dense_activation	optimizer_choice	n_layers	momemtum_range	loss_function	Score
# 274	            0.1	    he_uniform	        96	        0.00001	      linear	            adamw	            3	        0.7	            mse	      0.005235
# 384	            0.1	    he_uniform	        107	        0.00001	      None	              adamw	            5	        0.5	            mse	      0.005237

#For 120-30 horizon
# 294 	0.0 	0.4 	0.3 	he_uniform 	36 	0.00001 	None 	adamw 	2 	0.3 	mse 	0.004750
# 484 	0.1 	0.4 	0.3 	he_uniform 	38 	0.00001 	linear 	adamw 	2 	0.8 	mse 	0.004791
# 404 	0.1 	0.2 	0.2 	he_uniform 	44 	0.00001 	linear 	adamw 	2 	0.4 	mse 	0.004843

#For 120-30 horizon with linear learning rate
# 454 	0.2 	0.1 	0.3 	he_uniform 	56 	0.000201 	None 	adamw 	4 	0.7 	mse 	0.004770
# 194 	0.2 	0.0 	0.4 	he_uniform 	35 	0.000701 	linear 	adamw 	3 	0.4 	mse 	0.004792
# 304 	0.1 	0.0 	0.2 	he_uniform 	123 	0.000451 	None 	adamw 	3 	0.2 	mse 	0.004807

In [ ]:
#params['dropout'] = 0.2
#params["dropout_enc_0"] = 0.0
#params["dropout_dec_0"] = 0.2
#params['batch_size'] = 60
#params['hidden_units'] = 60
#params['learning_rate'] = 0.001
#params['optimizer_choice'] = 'adam'

'''params = {
    'hidden_units':          274,
'dropout':                      0.1,
'kernel_initializer':    'he_uniform',
'batch_size':                   107,
'learning_rate':              0.00001,
'dense_activation':          'linear',
'optimizer_choice':         'adamw',
'momemtum_range':               0.7,
'loss_function':              'mse',
'dropout_enc_0':                0.0,
'dropout_dec_0':                0.3,
'n_layers':                     4
}'''

print(params)

In [ ]:
def encoder_decoder_model(params):

  tf.keras.backend.clear_session()
  hidden_units = params.get('hidden_units')
  n_layers = params.get('n_layers')
  dense_activation = 'linear'
  loss_func_choice = params.get('loss_function')
  learning_rate = params.get('learning_rate')
  encoder_output = []
  kernel_initializer = params.get('kernel_initializer')
  input = Input(shape=X_train.shape[-2:])
  if n_layers > 1:
    enc_out = LSTM(units=hidden_units,
                      return_state=True,
                      return_sequences=True,
                      dropout=params.get('dropout_enc_0'),
                      kernel_initializer = kernel_initializer,
                      use_bias=True,
                      unit_forget_bias=params.get("enc_lstm_0_unit_forget_bias"),
                      name=f"encoder_lstm_0")(input)
    encoder_output.append(enc_out)
    # for i in np.arange(1, hp.get('n_layers')-1):
    for i in np.arange(1, n_layers-1):
      enc_out = ( LSTM(units=hidden_units,
                  return_state=True,
                  return_sequences=True,
                  #dropout=params.get(f'dropout_enc_{i-1}'),
                  dropout= 0.0,
                  # kernel_initializer = hp.Choice(f'kern_init_enc_lstm_{i}', ["glorot_uniform", 'he_uniform']),
                  kernel_initializer = kernel_initializer,
                  use_bias=True,
                  unit_forget_bias=params.get(f"enc_lstm_{i}_unit_forget_bias"),
                  name=f"encoder_lstm_{i}")(encoder_output[i-1][0] )
                )
      encoder_output.append(enc_out)
    enc_out = LSTM(units=hidden_units,
                    return_state=True,
                    #dropout=params.get(f'dropout_enc_{n_layers-1}'),
                    dropout = 0.0,
                  #  kernel_initializer = hp.Choice(f'kern_init_enc_lstm_{n_layers-1}', ["glorot_uniform", 'he_uniform']),
                    kernel_initializer = kernel_initializer,
                    use_bias=True,
                    unit_forget_bias=params.get(f"enc_lstm_{n_layers-1}_unit_forget_bias"),
                    name=f"encoder_lstm_{n_layers-1}")(encoder_output[-1][0])
    encoder_output.append(enc_out)
  else:
    enc_out = LSTM(units=hidden_units,
                    return_state=True,
                    dropout=params.get('dropout_enc_0'),
                    kernel_initializer = kernel_initializer,
                    use_bias=True,
                    unit_forget_bias=params.get(f"enc_lstm_0_unit_forget_bias"),
                    name=f"encoder_lstm_{0}")(input)
    encoder_output.append(enc_out)
  repeat = RepeatVector(horizon)
  dec_in = repeat(encoder_output[-1][0])
  dec_out = LSTM(units=hidden_units,
                  return_sequences=True,
                  dropout=params.get('dropout_dec_0'),
                  kernel_initializer = kernel_initializer,
                  use_bias=True,
                  unit_forget_bias=params.get(f"dec_lstm_0_unit_forget_bias"),
                  name='decoder_lstm_0')(dec_in,initial_state=encoder_output[0][1:])
  dec_out = tf.keras.layers.ReLU()(dec_out)
  if n_layers > 1:
    # for i in np.arange(1, hp.get('n_layers')):
    for i in np.arange(1, n_layers-1):
      dec_out = LSTM(units=hidden_units,
                    return_sequences=True,
                    #dropout=params.get(f'dropout_dec_{i}'),
                    dropout = 0.0,
                    use_bias=True,
                    unit_forget_bias=params.get(f"dec_lstm_{i}_unit_forget_bias"),

                    kernel_initializer = kernel_initializer,
                    name=f"decoder_lstm_{i}")(dec_out, initial_state=encoder_output[i][1:])
      dec_out = tf.keras.layers.ReLU()(dec_out)

  if dense_activation == "linear":
    dense = TimeDistributed(Dense(features, activation='linear'))
  final_out = dense(dec_out)
  model = Model(input, final_out)
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss=loss_func_choice,
                # optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_range),
                optimizer=optimizer,
                metrics=['mse',tf.keras.metrics.RootMeanSquaredError()])
  return model

In [ ]:
model = encoder_decoder_model(params)
model.summary()

In [ ]:
train_set, val_set = prepare_tensors(train=(X_train, y_train), validation=(X_val, y_val), batch_size=params.get('batch_size'), shuffle_buffer_size=shuffle_buffer_size)

In [ ]:
history = model.fit(train_set, validation_data = val_set, epochs=100)

In [ ]:
#model.save('/content/drive/MyDrive/marjan_multi_step_28Nov_all_metric_120_30step.h5')

In [ ]:
#model = tf.keras.models.load_model('/content/drive/MyDrive/marjan_single_step_26_09_2023')

In [ ]:
predictions = model.predict(X_test)
predictions.shape

In [ ]:
def performance_metrics(col_names, preds, true_val_, ts_date):
  test_size = preds.shape[0]
  true_val = true_val_.loc[ts_date.reshape(-1)][:test_size].reset_index()
  final_out = defaultdict(dict)
  date_col = ts_date.reshape(-1)[:test_size]
  for feat in col_names:
    rmse = mean_squared_error(y_true=true_val[feat], y_pred=preds[feat], squared=False)
    mae = mean_absolute_error(y_true=true_val[feat], y_pred=preds[feat])
    r2_sc = r2_score(y_true=true_val[feat], y_pred=preds[feat])
    mape = mean_absolute_percentage_error(y_true=true_val[feat], y_pred=preds[feat])
    print(f"{feat}\nMAE: {mae}\tRMSE: {rmse}\tR2 Score: {r2_sc}\tMAPE: {mape}\n")
    df = pd.DataFrame(data=[date_col, preds[feat]]).T
    df = df.rename(columns={0:'timestamp',1:feat})
    np_df = df.to_numpy()
    eval_metrics = {"rmse":rmse, "mae":mae, "r2_score":r2_sc, "mape": mape}
    metric_name = feat
    meta_d = {"metric": eval_metrics, 'values': np_df, 'best_prediction':False}
    final_out[metric_name] = meta_d

  return final_out

In [ ]:
print(type(X_test), X_test.shape)
#print(type(preds), preds.shape)
print(type(predictions), predictions.shape)
print(type(ts_dates), np.shape(ts_dates))

In [ ]:
test_data = test_data.reset_index()
print(type(test_data), test_data.shape)
print(type(ts_dates), ts_dates.shape)
test_data.head()

### Testing with multistep

In [ ]:
#unscale the predictions
print(predictions.shape[0])
predictions_ = []
X_test_ = []
y_test_ = []
for i in range(predictions.shape[0]):
  p1 = ts_scaler.inverse_transform(predictions[i,:,:])
  x1 = ts_scaler.inverse_transform(X_test[i,:,:])
  y1 = ts_scaler.inverse_transform(y_test[i,:,:])
  predictions_.append(p1)
  X_test_.append(x1)
  y_test_.append(y1)

predictions_ = np.array(predictions_)
X_test_ = np.array(X_test_)
y_test_ = np.array(y_test_)
print(type(predictions_), np.shape(predictions_))
print(type(X_test_), np.shape(X_test_))
print(type(y_test_), np.shape(y_test_))

In [ ]:
print(predictions.shape, np.shape(predictions[0,:,:]))
print(X_test.shape, np.shape(X_test[0,:,:]))
#preds = predictions[0,:,:]
#trues = X_test[0,:,:]
#trues_y = y_test[0,:,:]
lst = [i+window_size for i in range(horizon)]
count = 0
for i,j in enumerate(range(0,predictions_.shape[0],50)):
  count+=1
  #print("i=", i, "j=", j)
print("count=", count, round(count/3))
figure, ax = plt.subplots(round(count/3), 3, figsize=(15,60))
for j, i in enumerate(range(0,predictions_.shape[0],50)):
  #print("i=", i)
  ax = ax.ravel()
  #ax[0][0].set_xlim(0,310)
  forecast_values = predictions_[i,:,0]
  dataframe_cols = {"yhat": np.array(forecast_values)}

  upper_bound = np.array(
            [
                (
                        forecast_values[i] + (np.std(forecast_values[:i]) * 3)
                )
                for i in range(len(forecast_values))
            ]
        )
  upper_bound[0] = np.mean(
            forecast_values[0]
        )  # to account for no std of a single value
  lower_bound = np.array(
            [
                (
                        forecast_values[i] - (np.std(forecast_values[:i]) * 3)
                )
                for i in range(len(forecast_values))
            ]
        )
  lower_bound[0] = np.mean(
            forecast_values[0]
        )  # to account for no std of a single value
  ub = upper_bound
  lb = lower_bound
  #fig, ax = plt.subplots(1)
  #print("shape:", np.shape(predictions_[i,:,:]), np.shape(predictions_))
  ax[j].plot(X_test_[i,:,0], label='GroundTruth')
  ax[j].plot(lst, predictions_[i,:,0], label='Prediction for {} metric'.format(multi_data.columns[0]))
  ax[j].plot(lst, y_test_[i,:,0], label='True_y')
  ax[j].plot(lst, ub, label='Upper Bound')
  ax[j].plot(lst, lb, label='Lower Bound'),
  ax[j].fill_between(lst, lb, ub, facecolor='bisque', alpha=0.5,
                label='std-dev range')
  #plt.set_title(test_data.columns[1])
  plt.suptitle('Groundtruth and predicted plots for {}'.format(multi_data.columns[0]))
  plt.tight_layout()
plt.legend()
#plt.show()

In [ ]:
print(predictions.shape, np.shape(predictions[0,:,:]))
print(X_test.shape, np.shape(X_test[0,:,:]))
#preds = predictions[0,:,:]
#trues = X_test[0,:,:]
#trues_y = y_test[0,:,:]
for met in range(len(multi_data.columns)):
  lst = [i+window_size for i in range(horizon)]
  count = 0
  for i,j in enumerate(range(0,predictions_.shape[0],50)):
    count+=1
    #print("i=", i, "j=", j)
  print("count=", count, round(count/3))
  in_range_pred = []
  figure, ax = plt.subplots(round(count/3), 3, figsize=(15,60))
  for j, i in enumerate(range(0,predictions_.shape[0],50)):
    ax = ax.ravel()
    #ax[0][0].set_xlim(0,310)
    forecast_values = predictions_[i,:,met]
    dataframe_cols = {"yhat": np.array(forecast_values)}
    upper_bound = np.array(
            [
                (
                        forecast_values[i] + (np.std(forecast_values[:i]) * 2)
                )
                for i in range(len(forecast_values))
            ]
        )
    upper_bound[met] = np.mean(
            forecast_values[met]
        )  # to account for no std of a single value
    lower_bound = np.array(
            [
                (
                        forecast_values[i] - (np.std(forecast_values[:i]) * 2)
                )
                for i in range(len(forecast_values))
            ]
        )
    lower_bound[met] = np.mean(
            forecast_values[met]
        )  # to account for no std of a single value
    ub = upper_bound
    lb = lower_bound
    #fig, ax = plt.subplots(1)
    #print("shape:", np.shape(predictions_[i,:,:]), np.shape(predictions_))
    #if lb <= y_test_[i,:,0] <= ub:
    outliercount = 0
    valid_count = 0
    for tv, lower, upper in zip(y_test_[i,:,met], lb, ub):
      if lower <= tv <= upper:
        valid_count+=1
      else:
        outliercount+=1


    ax[j].plot(X_test_[i,:,met], label='GroundTruth')
    ax[j].plot(lst, predictions_[i,:,met], label='Pred- {} metric'.format(multi_data.columns[met][-30:]))
    ax[j].plot(lst, y_test_[i,:,met], label='True_y::Outliers={}'.format(outliercount))
    ax[j].plot(lst, ub, label='Upper Bound')
    ax[j].plot(lst, lb, label='Lower Bound'),
    ax[j].fill_between(lst, lb, ub, facecolor='bisque', alpha=0.5,
                label='std-dev range::valid={}'.format(valid_count))
    #ax[j].ylabel(outlier_count)
    #plt.set_title(test_data.columns[1])
    in_range_pred.append(valid_count/horizon)
    plt.suptitle('Groundtruth and predicted plots for {}'.format(multi_data.columns[met]))
    plt.tight_layout()
    ax[j].legend()
  print("Score for metric {} is :{}".format(multi_data.columns[met],sum(in_range_pred)/len(in_range_pred)))

In [ ]:
# Do training on mean(5min resample) -- train on all 6 metric
multi_data.columns[0][-30:]